In [ ]:
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import ttk
import time
import threading

# --- Глобальные параметры ---
WIDTH, HEIGHT = 400, 300
UPDATE_EVERY_N_PIXELS = 500

# --- Глобальные переменные состояния ---
running = False
img = None
pixels = None
tk_img = None
start_time = None
update_counter = 0
render_thread = None

# --- Функция отрисовки (в отдельном потоке) ---
def render_loop(canvas, progress_bar, time_label, speed_scale):
    global running, img, pixels, tk_img, start_time, update_counter

    img = Image.new("RGB", (WIDTH, HEIGHT), "black")
    pixels = img.load()

    start_time = time.time()
    total_pixels = WIDTH * HEIGHT
    pixel_index = 0

    for y in range(HEIGHT):
        if not running:
            break
        for x in range(WIDTH):
            if not running:
                return

            delay = speed_scale.get()
            time.sleep(delay / 1000.0)

            # Градиент
            r = int((x / WIDTH) * 255)
            g = int(((x + y) / (WIDTH + HEIGHT)) * 100)
            b = int((y / HEIGHT) * 255)
            pixels[x, y] = (r, g, b)

            pixel_index += 1
            update_counter += 1

            if update_counter >= UPDATE_EVERY_N_PIXELS:
                # Обновляем изображение
                new_tk_img = ImageTk.PhotoImage(img)
                canvas.config(image=new_tk_img)
                canvas.image = new_tk_img  # Сохраняем ссылку!

                # Обновляем прогресс и таймер
                progress = (pixel_index / total_pixels) * 100
                progress_bar['value'] = progress
                elapsed = time.time() - start_time
                time_label.config(text=f"Время: {elapsed:.1f} сек")

                # Принудительно обновляем виджеты
                canvas.update_idletasks()
                progress_bar.update_idletasks()
                time_label.update_idletasks()

                update_counter = 0

    # Финал
    final_tk_img = ImageTk.PhotoImage(img)
    canvas.config(image=final_tk_img)
    canvas.image = final_tk_img
    progress_bar['value'] = 100
    elapsed = time.time() - start_time
    time_label.config(text=f"✅ Готово за {elapsed:.1f} сек")
    img.save("slow_render_final.png")
    print("Изображение сохранено как 'slow_render_final.png'")

# --- Запуск рендеринга ---
def start_render(canvas, progress_bar, time_label, speed_scale, start_btn, stop_btn):
    global running, render_thread

    if running:
        return

    running = True
    start_btn.config(state="disabled")
    stop_btn.config(state="normal")
    progress_bar['value'] = 0
    time_label.config(text="Время: 0.0 сек")

    render_thread = threading.Thread(
        target=render_loop,
        args=(canvas, progress_bar, time_label, speed_scale),
        daemon=True
    )
    render_thread.start()

# --- Остановка ---
def stop_render(start_btn, stop_btn):
    global running
    running = False
    start_btn.config(state="normal")
    stop_btn.config(state="disabled")
    print("Рендеринг остановлен пользователем.")

# --- Создание интерфейса ---
def create_gui():
    root = tk.Tk()
    root.title("🎨 Медленный CPU-рендеринг (GPU отсутствует)")
    root.geometry("500x700")  # Увеличим высоту окна
    root.resizable(True, True)  # Разрешаем изменение размера

    # Создаём Frame для холста с фиксированным размером
    canvas_frame = tk.Frame(root, width=WIDTH, height=HEIGHT, bg="gray")
    canvas_frame.pack(pady=10)
    canvas_frame.pack_propagate(False)  # Отключаем авто-подстройку размера

    # Холст (Label) внутри Frame
    canvas = tk.Label(canvas_frame, bg="black")
    canvas.pack(expand=True, fill="both")

    # Прогресс-бар
    tk.Label(root, text="Прогресс:").pack(pady=(10,0))
    progress_bar = ttk.Progressbar(root, length=400, mode='determinate')
    progress_bar.pack(pady=5)

    # Таймер
    time_label = tk.Label(root, text="Время: 0.0 сек", font=("Arial", 12))
    time_label.pack(pady=5)

    # Ползунок скорости
    tk.Label(root, text="Скорость (чем меньше — тем медленнее):").pack(pady=(20,5))
    speed_scale = tk.Scale(root, from_=1, to=100, orient="horizontal", length=400, resolution=1)
    speed_scale.set(5)
    speed_scale.pack(pady=5)
    tk.Label(root, text="1 = быстро, 100 = очень медленно").pack()

    # Кнопки
    btn_frame = tk.Frame(root)
    btn_frame.pack(pady=20)

    start_btn = tk.Button(btn_frame, text="▶️ Старт", font=("Arial", 14), width=10,
                          command=lambda: start_render(canvas, progress_bar, time_label, speed_scale, start_btn, stop_btn))
    start_btn.pack(side="left", padx=10)

    stop_btn = tk.Button(btn_frame, text="⏹️ Стоп", font=("Arial", 14), width=10,
                         state="disabled",
                         command=lambda: stop_render(start_btn, stop_btn))
    stop_btn.pack(side="left", padx=10)

    # Подсказка
    tk.Label(root, text="Имитация рендеринга без GPU — каждый пиксель рисуется вручную 😅", fg="gray").pack(side="bottom", pady=10)

    root.mainloop()

# --- Запуск ---
if __name__ == "__main__":
    create_gui()